In [1]:
from src.learner import Learner as L
import numpy as np

# inputs and outputs
X = np.genfromtxt('data/orth.csv', delimiter=",")
Y = np.genfromtxt('data/phon.csv', delimiter=",")

2024-01-12 14:13:12.116568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-12 14:13:12.116597: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
L.train(orthography=X, phonology=Y)

TypeError: train() missing 1 required positional argument: 'self'

(2881, 364)